In [ ]:
import chargeback_rpt.db_postgres_command as db_command
import chargeback_rpt.veeam_data_extractor as rpt_veeam
import chargeback_rpt.vm_data_utility as vm_util
import chargeback_rpt.vm_data_validator as vx
import chargeback_rpt.vm_data_charger as vc
import chargeback_rpt.email_notifier as x_mail
import chargeback_rpt.file_directory_manager as fd_mn
import pandas as pd
import numpy as np

from pytz import timezone
import datetime
import os

import openpyxl

import psycopg2
import psycopg2.extras as extras


import json
import sys 

# update 23/12/22 filter exclude 


# For run script .py

In [ ]:
run_py=False

print("Update 23/12/22")
print("1.remove vCLS-")
print("2.change if/else statement : List VM-Off contain terminated_date")

is_only_check_data=False  # Check data=True , ETL=False
use_backup=False   # Use Veeam=True, Disable veem=False

fix_mmyy=False  #fix_mmyy=True  for test_only_jupyternotebook

veeam_ok=False
vdisk_ok=False
vinfo_ok=False

veeam_filepath=None
disk_filepath=None
info_filepath=None

# disable veeam  
if is_only_check_data==True:
 use_backup=False  # Use Veeam=True, Disable veem=False
 veeam_ok=True


x_datenow=datetime.datetime.now()
month_x=x_datenow.strftime('%m')
year_x=x_datenow.strftime('%Y')

if fix_mmyy:
 month_param='09'
 year_param='2021'
else:
 month_param=month_x
 year_param=year_x

print(f"Current month-year : {month_param}-{year_param}")

In [ ]:
# sys.argv[1]=month  , sys.argv[2]=year ,  sys.argv[3]=is_check_data such as 0=check_only , 1=actual run

if  run_py:

    press_y='n'
    ok=False
    
    if len(sys.argv) > 1:
        if sys.argv[1]=='0'  and sys.argv[2]=='0':
            
         month_param=month_x
         year_param=year_x 
         ok=True   
         is_only_check_data=bool(int(sys.argv[3]))
            
        else:
         raise Exception("Specify 0 0 as arguments only")   
    else:
        print("If you want current month and year.Press Enter ")
        month_param = int(input("Enter the month (1-12) : ") or month_x)
        year_param = int(input("Enter the year(such as 2021,2022,2023...) : ") or year_x)
        print(f"Do for month={month_param} and year={year_param}")
        press_y=input(f"Press y=True and n=False : ") 
        if press_y.lower()=='y':
         ok=True


    if ok==True:
        try:
         report_dt= datetime.datetime(int(year_param), int(month_param), 1)
         print(report_dt)   
         month_param=report_dt.strftime('%m')
         year_param=report_dt.strftime('%Y')   

    #      n = len(sys.argv) 
    #      print("Total arguments passed:", n) 
    #      for  param in  sys.argv:
    #        print(param)

        except Exception as ex:
         print(ex)   
         raise ex
    else:
         quit()


In [ ]:
if is_only_check_data==False:
 t_id=vm_util.creating_transaction(1,month_param,year_param)
 
else:
  t_id=vm_util.creating_transaction(8,month_param,year_param)   
    
print(f"ETL Transaction ID: {t_id}  and OnlyCheckData={is_only_check_data}")    

In [ ]:
list_error=[]
print(list_error)

In [ ]:
# Move  to util
def check_error_point_etl(tran_id):
  print(list_error)
  if True in list_error:
        
    etl_files=[]
# disable veeam    
#     if  veeam_filepath is not None and os.path.exists(veeam_filepath):
#      etl_files.append([veeam_ok,veeam_filepath])

    if  (disk_filepath is not None )  and (os.path.exists(disk_filepath)):
     etl_files.append([vdisk_ok,disk_filepath])  
    if (info_filepath is not None) and (os.path.exists(info_filepath)):
     etl_files.append([vinfo_ok,info_filepath])     
    
    if len(etl_files)>0 and ( (veeam_ok==False) or (vdisk_ok==False)  or (vinfo_ok==False) ):
       vm_util.finished_etl_folder(t_id,vm_col_key,False,etl_files)  
    
    vm_util.collect_error_to_sent_mail(tran_id)  
    print("ETL occured error") 
    
    raise Exception("Program is teminated and check error from email and log_error.txt")
    
  list_error.clear()
  print(list_error)   

In [ ]:
vm_tb_key="vm_table"
vm_col_key='vm'

try:
 unknow_val=vm_util.get_config_value("unknow",t_id)
 datetime_format = vm_util.get_config_value("datetime_format", t_id)
except:
 list_error.append(True)   
 print(error_message)   
    

update_additional_cost_key='email_link_update_additional_cost'
all_additional_cost_key='email_link_all_additional_cost'



In [ ]:
check_error_point_etl(t_id)

# Verify all source files

In [ ]:
print("get root folder")

key_path='import_path'
import_path=vm_util.get_value_by_key(key_path)
print(import_path)

if import_path is not None:
 
 data_import_path=import_path['value']
 existPath,is_error=vx.check_existing_filepath(data_import_path,t_id)
 list_error.append(is_error)   
 if existPath:
  print("Get Path of all source files: ",data_import_path)
else:
 list_error.append(True)   
 error_message= f'no key:{key_path} in key column in config_value table'
 print(error_message)   
 vm_util.add_error_to_database(3,error_message,t_id)  

In [ ]:

check_error_point_etl(t_id)


In [ ]:
print("get source file name for ready to do ETL")

sr_veeam_ds,is_error=vx.check_source_name_file('veeamone',t_id)
print(sr_veeam_ds)
list_error.append(is_error)

sr_disk_ds,is_error=vx.check_source_name_file('rvtools_disk',t_id)
print(sr_disk_ds)
list_error.append(is_error)

sr_datasource,is_error=vx.check_source_name_file('rvtools_info',t_id)
print(sr_datasource)
list_error.append(is_error)


In [ ]:

check_error_point_etl(t_id)


In [ ]:
print("check file path for doing ETL")
def check_existing_file_path(path,is_onefile_folder) :
    
 if is_onefile_folder==True:
   try:  # new version
    result=vm_util.extract_single_file_as_specified_name_in_folder(path,t_id) 
    print(f'{path} = {result}')
   except Exception as error:
    list_error.append(True)    
 else:  # check version1
    is_existing,is_error=vx.check_existing_filepath(path,t_id)  
    print(f'{path} = {is_existing}')
    list_error.append(is_error)
    
print("=========================================================================") 
if is_only_check_data==False:  
    print("check Path : VeeamOne")
    print(sr_veeam_ds)
    veeam_filepath=f"{data_import_path}\\{sr_veeam_ds['source_prefix']}.{sr_veeam_ds['source_type']}"
    # disable veeam
    #check_existing_file_path(veeam_filepath,sr_veeam_ds['is_onefile_folder'] )
print("=========================================================================")   

print("check Path : VMDisk")
print(sr_disk_ds)
disk_filepath=f"{data_import_path}\\{sr_disk_ds['source_prefix']}.{sr_disk_ds['source_type']}"
check_existing_file_path(disk_filepath,sr_disk_ds['is_onefile_folder'] )
print("=========================================================================")  

print("check Path : VMInfo")
print(sr_datasource)
info_filepath=f"{data_import_path}\\{sr_datasource['source_prefix']}.{sr_datasource['source_type']}"
check_existing_file_path(info_filepath,sr_datasource['is_onefile_folder'] )
print("=========================================================================")  

In [ ]:

check_error_point_etl(t_id)


# Load Veeam-One Data

In [ ]:
if use_backup==True:
    try: 
     print("load veeam-one files")
     df_veeam_mappingFields=vm_util.get_active_datafield(sr_veeam_ds['id'])
     print(df_veeam_mappingFields)

     df_veeam=rpt_veeam.extract_veeam_report_data(veeam_filepath,df_veeam_mappingFields)
     if df_veeam is not None:
      print(df_veeam.info())      
      print(df_veeam.head(10))
      print(df_veeam.isna().sum())  

      dict_veeam_mapplingCols=dict(zip(df_veeam_mappingFields['field_source_name'],df_veeam_mappingFields['column_table_name']))
      print(dict_veeam_mapplingCols)
      df_veeam.rename(columns=dict_veeam_mapplingCols,inplace=True) 
      print(df_veeam.head(20))


     if df_veeam is None:
        raise  Exception("No any backup policy in Veeam")
     else:
        list_error.append(False)  


      #df_veeam.to_excel('veeam_temp.xlsx',index=False)
     #=========================================================
    except Exception as error:
     list_error.append(True)   
     vm_util.add_error_to_database(6,str(error),t_id)
     print('error is ',str(error))

# disable veeam
else:
    print(f"No need veeam file path due to only check data  OR disabled veeam")
#     df_veeam=pd.DataFrame(data=list())
#     print(df_veeam)


In [ ]:
check_error_point_etl(t_id)
veeam_ok=True

# Load vinfo and vdisk Data

In [ ]:
print("load vinfo and vdisk files")

df_vDisk,is_error=vx.load_source_file(disk_filepath,sr_disk_ds['source_keyname'],t_id)
list_error.append(is_error)

df_vInfo,is_error=vx.load_source_file(info_filepath,sr_datasource['source_keyname'],t_id)
list_error.append(is_error)
 
check_error_point_etl(t_id) 
vdisk_ok=True

In [ ]:
print(df_vDisk.head(5))
print(df_vInfo[['VM','CreatedDate']].head(5))

# Selec required columns from dataframe


In [ ]:
print("Selec required columns from dataframe")

print("select field in disk info")
df_disk_mappingFields=vm_util.get_active_datafield(sr_disk_ds['id'])
print("==================vDisk mappingFields==================================")
print(df_disk_mappingFields[['column_table_name','field_source_name','is_additional','datasource_id']])
df_vDisk,err=vx.select_colunm_df_fieldname_ds(df_vDisk,df_disk_mappingFields['field_source_name'].tolist(),disk_filepath,t_id)
list_error.append(err)

print("=====================================================================================================")


print("select field in vm info")
df_mappingFields=vm_util.get_active_datafield(sr_datasource['id'])
df_extra_mappingFields=df_mappingFields.query('is_json_additional_info==True')


print("==================vIfno mappingFields==================================")
print(df_mappingFields[['column_table_name','field_source_name','is_additional','is_not_null','datasource_id']])
print("==================vIfno additional mappingFields=========================================")
print(df_extra_mappingFields[['column_table_name','field_source_name','is_additional','is_json_additional_info','datasource_id']])

df_vInfo,err=vx.select_colunm_df_fieldname_ds(df_vInfo,df_mappingFields['field_source_name'].tolist(),info_filepath,t_id)
list_error.append(err)

check_error_point_etl(t_id)


In [ ]:
print(df_vDisk.info())
print(df_vInfo.info())

# Mapping filed from datasource to column dataframe

In [ ]:
print("Mapping filed from datasouce to column dataframe")

In [ ]:
print("Mapping filed Disk info")
df_vDisk,err=vx.map_fields_ds_to_cols_df(df_vDisk,df_disk_mappingFields,disk_filepath,t_id)
list_error.append(err)
print("Mapping filed VM info")
df_vInfo,err=vx.map_fields_ds_to_cols_df(df_vInfo,df_mappingFields,info_filepath,t_id)
list_error.append(err)

check_error_point_etl(t_id)

# Filter VM Info and Check Duplicate VM and Remove SystemVM(Reboot)

In [ ]:
# load from config database
strExVM="vCLS-,TestTemp"
exVM=tuple(strExVM.split(','))
print("Exclude VM : ")
df_vInfo=df_vInfo[~df_vInfo.vm.str.startswith((exVM))]
print(df_vInfo.shape)

In [ ]:
print("Filter VM Info and Check Duplicate VM")


isNoteTemplate = False 
df_vInfo=df_vInfo.query("template==@isNoteTemplate")

#vm_state= 'poweredOn'
#df_vInfo=df_vInfo.query("powerstate==@vm_state  and  template==@isNoteTemplate and primary_ip_address==primary_ip_address") 
#df_vInfo=df_vInfo.query("template==@isNoteTemplate and primary_ip_address==primary_ip_address")

if len(df_vInfo.index)==0:
  list_error.append(True)
  error_message= "no vm data"
  print(error_message)   
  vm_util.add_error_to_database(11,error_message,t_id)  
                       

check_error_point_etl(t_id)

df_dup,is_error=vx.find_SameVMName(df_vInfo,'vm')
if df_dup is not None:
  list_error.append(is_error)
  error_message= "found some same vm name\n"
  error_message+=df_dup.to_html()  
  vm_util.add_error_to_database(13,error_message,t_id)  

check_error_point_etl(t_id)

# Check Null Value

In [ ]:
print("Check Null Value")

#not check veeam-one because data is got rid of all of null veluse to be 0  extract_veeam_report_data zero 
vdisk_notNaN_cols=(df_disk_mappingFields.query('is_not_null==True')['column_table_name']).tolist()
vinfo_notNaN_cols=(df_mappingFields.query('is_not_null==True')['column_table_name']).tolist()

print('list all column must be not null')
print(vdisk_notNaN_cols)
print(vinfo_notNaN_cols)


In [ ]:
srDisk_NaN,dfDisk_NaN=vx.find_NaN(df_vDisk,vdisk_notNaN_cols)
str_diskNaN_error,diskNaN_error=vx.report_ErrorValues(srDisk_NaN,dfDisk_NaN[['vm_disk']+vdisk_notNaN_cols],t_id,"Null",8)
list_error.append(diskNaN_error)

In [ ]:
srVM_NaN,dfVM_NaN=vx.find_NaN(df_vInfo,vinfo_notNaN_cols) 
str_error,x_error=vx.report_ErrorValues(srVM_NaN,dfVM_NaN[['vm']+vinfo_notNaN_cols],t_id,"Null",8)
list_error.append(x_error)

In [ ]:
check_error_point_etl(t_id)

# Merge Veeam and Disk to VM Info

# Merge Veeam to VM

In [ ]:
try:
    
 if (is_only_check_data==False) and (use_backup==True):   
   print("Merge Veeam  to VM Info") 
   df_vInfo=df_vInfo.merge(df_veeam[['VM','backup_size_gb']],how='outer',left_on='vm',right_on='VM')
   df_vInfo['vm']=df_vInfo.apply(lambda x: x['VM'] if vx.isnan(x['vm']) else x['vm'] ,axis=1)
   df_vInfo.drop(columns='VM',inplace=True)
 else:
   df_vInfo['backup_size_gb']=0
 
except Exception as err:
 list_error.append(True)
 error_message= str(err)
 print(error_message)   
 vm_util.add_error_to_database(14,error_message,t_id) 
 



# Filter & Transform Disk Info before Merge


In [ ]:
print("Group disks before merge")
df_vDisk=df_vDisk.query('template_disk==False')
print("Sum total capacity by vm-id")
df_vDisk=df_vDisk.groupby('vm_disk_id',as_index=False).sum()
print(df_vDisk.head(20))

# Merge vDisk to VM

In [ ]:
print("Merg Disk to VM Info")
try:
 df_vInfo=df_vInfo.merge(df_vDisk[['vm_disk_id','capacity_gb']],how='left',left_on='vm_id',right_on='vm_disk_id')
 df_vInfo.drop(columns='vm_disk_id',inplace=True)
    
 # replace  numberic joined value with 0 
 df_vInfo= df_vInfo.replace( {'backup_size_gb':{np.nan:0}, 'capacity_gb':{np.nan:0} })   
  
except Exception as err:
 list_error.append(True)
 error_message= str(err)
 print(error_message)   
 vm_util.add_error_to_database(14,error_message,t_id) 
    
    


In [ ]:
print("Data after merge")
print(df_vInfo.info())
print(df_vInfo[['vm','vm_id','cpu','backup_size_gb','backup_size_gb','os','database','system_name','cost_center','created_date','terminated_date']].head(10))

#df_vInfo.to_excel('vinfo.xlsx')


# Fill VM data for OnlyBackup

In [ ]:
print("Fill VM data for OnlyBackup")


numbericCols = vc.list_master_cost()
numColsList = numbericCols['column_table_name'].tolist()
nonNumeric_cols = [x for x in df_vInfo.columns.tolist() if x in numColsList]
#nonNumeric_cols


backup_size_colName='backup_size_gb'
notCopyCol=[backup_size_colName]

numericColToSetDefaultVal=nonNumeric_cols
numericColToSetDefaultVal.remove(backup_size_colName)

objectColToSetUnknown=vinfo_notNaN_cols.copy()
objectColToSetUnknown=[x for x in  objectColToSetUnknown if  x not in numericColToSetDefaultVal]
objectColToSetUnknown.remove('powerstate')
objectColToSetUnknown.remove('vm')

print(numericColToSetDefaultVal)
print(objectColToSetUnknown)

#----------------------------------------------------------------------------------------

# create columne only backup first
df_vInfo['only_backup']  =False

print('No.row of vInfo DF before spliting=',df_vInfo.shape[0])

# splite backup datafraom from main dataframe 
df_none_vm=df_vInfo[df_vInfo['vm_id'].isna()].copy()
df_vInfo.drop(df_none_vm.index , inplace=True)

print('No.row of vInfo DF after spliting=',df_vInfo.shape[0])
print('No.row of noneVM DF after spliting=',df_none_vm.shape[0])

#----------------------------------------------------------------------------------------


if df_none_vm.shape[0]>0:

 print('These are only backup data') 
 df_none_vm['only_backup']  =True
 df_none_vm['powerstate']  =unknow_val
 df_none_vm[numericColToSetDefaultVal]=0.0

 print(df_none_vm[['vm','vm_id','cpu','memory','capacity_gb',backup_size_colName,'os','database','system_name','cost_center']])
 
 list_onlybackup_df=[]


 tb_sr = vm_util.get_value_by_key(vm_tb_key)
 if tb_sr is not None:
   tb_name = tb_sr['value']

   excols_none_vm=df_none_vm.columns.tolist()
   #print(excols_none_vm)   
   for index, item  in df_none_vm.iterrows(): 
        
     # find the same vm inf as vm-name in backup 
     #found historical record 
     sr_prev_vm=vc.get_x_info(tb_name,vm_col_key,item['vm'])
     if  sr_prev_vm is not None:
     
      for blank_col in excols_none_vm:
       if (blank_col in  sr_prev_vm.index) and (blank_col in objectColToSetUnknown):
        #print('found object : ',excol)
        item[blank_col]= sr_prev_vm[blank_col]   
        
        
         
     else: #either set defaul value or None  
        
        for blank_col in excols_none_vm:
         if (blank_col not in numericColToSetDefaultVal) and (blank_col in objectColToSetUnknown ) :
           item[blank_col]=unknow_val
        
        item['created_date']=datetime.datetime(1900,1,1)
        
     # found backup but not found vm , we use init datetime  
     list_onlybackup_df.append(item.to_frame().transpose())  
   
 df_none_vm=None
 df_none_vm=pd.concat(list_onlybackup_df)

 print("VM found backup only after transforming data")
 print(df_none_vm[['vm','vm_id','cpu','memory','capacity_gb',backup_size_colName,'system_name','cost_center','created_date']]) 
   

else:
   print('No only backup data') 

# DateTime Validation

# Convert  datetime format

In [ ]:
import calendar
lastDay_val=datetime.date(int(year_param),int(month_param),calendar.monthrange(int(year_param), int(month_param))[1])
print(lastDay_val)

# Validate CreatedDate

In [ ]:
print("Validate CreatedDate")

df_vInfo=vx.find_incorrrect_format_for_input_datetime(df_vInfo,"created_date","vm",datetime_format,t_id)
if df_vInfo is None:
 list_error.append(True)
 print(list_error)
else:
 print(df_vInfo[['vm','created_date']] ) 

check_error_point_etl(t_id)


In [ ]:
df_vInfo=vx.find_invalid_x_date_greater_last_date_for_input_datetime(df_vInfo,"created_date",lastDay_val,"vm",t_id)
if df_vInfo is None:
 list_error.append(True) 
 print(list_error)
else:
 print(f"all created_date > {lastDay_val}")   
 print(df_vInfo[['vm','created_date']] ) 
    
check_error_point_etl(t_id)

# Validate Power On must be empty value in TerminatedDate

In [ ]:
print("Check poweredOn not allow to contain terminated_date.")
print("Check poweredOff  allow to not contain terminated_date but costing module will apply last day of month instead.")

# df_off_null = df_vInfo[ (df_vInfo['powerstate']=='poweredOff')  & (df_vInfo['terminated_date'].isnull())].copy()
# if  len(df_off_null.index)>0:
#    x=df_off_null[['vm','created_date','terminated_date','powerstate']]
#    vm_util.add_error_to_database(1,f"Error because some off-line vm contain null value in terminated_date.<br>{x.to_html(index=False)}",t_id)
#    print(x)
#    list_error.append(True)
#    print(list_error) 
    

df_on_not_null = df_vInfo[ (df_vInfo['powerstate']=='poweredOn')  & (df_vInfo['terminated_date'].notnull())].copy()
if  len(df_on_not_null.index)>0:
   x=df_on_not_null[['vm','created_date','terminated_date','powerstate']] 
   vm_util.add_error_to_database(29,f"Error because  some  online vm contain value in terminated_date.<br>{x.to_html(index=False)}",t_id) 
   print(x)
   list_error.append(True)
   print(list_error) 

check_error_point_etl(t_id)

# Validate PowerOff  and TerminatedDate

In [ ]:
print(f"Split VM-poweredOff data from main dataflow to ease to process condition")
dfx_off = df_vInfo[ (df_vInfo['powerstate']=='poweredOff')].copy()
print(dfx_off[['vm','created_date','terminated_date','powerstate']])

df_vmOff_nullTerminatedDate=dfx_off[dfx_off['terminated_date'].isnull() ].copy()
df_vmOff_notNullTerminatedDate=dfx_off[dfx_off['terminated_date'].notnull() ].copy()
print(df_vmOff_nullTerminatedDate)
print(df_vmOff_notNullTerminatedDate)

In [ ]:
# if len(df_vmOff_notNullTerminatedDate.index)>0 :  # update 31 Mary 22 on pandas 1.3 but origianl it is 1.1

print(f"VM-poweredOff Convert  terminated_date from string to datetime only row that contain value")
df_vmOff_notNullTerminatedDate=vx.find_incorrrect_format_for_input_datetime(df_vmOff_notNullTerminatedDate,"terminated_date","vm",datetime_format,t_id)
print(df_vmOff_notNullTerminatedDate)


In [ ]:

if df_vmOff_notNullTerminatedDate is not None and df_vmOff_notNullTerminatedDate.empty==False :
 print("List VM-Off contain terminated_date")   
 print(df_vmOff_notNullTerminatedDate[['vm','created_date','terminated_date','powerstate']])
 print(df_vmOff_notNullTerminatedDate.info())   


#  nan_xyz = np.nan
 print(f"Check error if  terminated_date >  created_date  or  terminated_date > {lastDay_val}")
 #dfx_off_error1=df_vmOff_notNullTerminatedDate.query("( terminated_date != @nan_xyz)   and ( created_date>terminated_date or terminated_date>@lastDay_val)")
 dfx_off_error1= df_vmOff_notNullTerminatedDate[ df_vmOff_notNullTerminatedDate['terminated_date'] \
                                                < df_vmOff_notNullTerminatedDate['created_date'] ]
 print(dfx_off_error1)

 if  len(dfx_off_error1.index)>0:
   x=dfx_off_error1[['vm','created_date','terminated_date','powerstate']] 
   print(x)
   vm_util.add_error_to_database(29,f"found one of the following errors  1.created_date > terminated_date 2.terminated_date > {lastDay_val}. 3.found terminated date of prev month despite existing VM {x.to_html(index=False)}",t_id) 

   list_error.append(True)   


 print(f"Check error if  terminated_date is not  in month==>{month_param}")
# run month 8  but terminated_date =7 ==> this vm have to be delterd since last month
#if existing  , this value must be empty or terminated_date = currunt month

 dfx_off_error2=df_vmOff_notNullTerminatedDate[ (df_vmOff_notNullTerminatedDate['terminated_date'] < datetime.datetime(int(year_param),int(month_param),1)) | (df_vmOff_notNullTerminatedDate['terminated_date'] >= datetime.datetime(int(year_param),int(month_param)+1,1)) ]
 print(dfx_off_error2)


 if dfx_off_error2.shape[0]>0:
   y=dfx_off_error2[['vm','created_date','terminated_date','powerstate']] 
   print(y)
   begin_date_str=(datetime.datetime( int(year_param),int(month_param),1)).strftime("%Y-%m-%d") 
   vm_util.add_error_to_database(29,f"found one of the following errors 1.created_date > terminated_date 2.terminated_date > {lastDay_val}(terminated_date of the future  month). 3.terminated_date < { begin_date_str}(terminated_date of the past month) {y.to_html(index=False)}",t_id)  
   list_error.append(True)

# else:
#     print("no vm specified terminated date")
          
check_error_point_etl(t_id)

In [ ]:
print("Merge vmOff_nullTerminatedDate with vmOff_notNullTerminatedDate")
dfx_off=pd.concat([df_vmOff_nullTerminatedDate,df_vmOff_notNullTerminatedDate]) 
dfx_off.reset_index(drop=True,inplace=True)
print(dfx_off[['vm','created_date','terminated_date','powerstate']])   
print(dfx_off.info())   
check_error_point_etl(t_id)


In [ ]:
print(f"Split poweredOn data from main dataflow to merge with poweredOff")
dfx_on=df_vInfo[ ~df_vInfo['vm'].isin(dfx_off['vm'].tolist())  ]
print(dfx_on[['vm','created_date','terminated_date','powerstate']])


print(f"Merge poweredOn with poweredOff to be complete data for building report")
df_vInfo=pd.concat([dfx_on,dfx_off])
df_vInfo.reset_index(drop=True,inplace=True)

print(df_vInfo.info())
print("VM Validation has been completed")
print(df_vInfo[['vm','created_date','terminated_date','powerstate']])

# Merge only backup to vm

In [ ]:
if df_none_vm.shape[0]>0:
 df_vInfo=pd.concat([df_vInfo,df_none_vm],ignore_index=True,axis=0)

check_error_point_etl(t_id)

print(df_vInfo.head()) 
print(df_vInfo.tail()) 

# Find Non-Numberic Data

In [ ]:
# Find Non-Numberic Data
print("Find Non-Numberic (Do after only back)")
srNN, dfNN,colsNN= vx.find_NonNumeric(df_vInfo)
print(colsNN)
print("the following columns are required numeric value" ,colsNN)
str_error, error=vx.report_ErrorValues(srNN,dfNN[['vm']+colsNN],t_id,"Non-Numeric",15)
#print(srNN)
#print(dfNN)

list_error.append(error)
print(str_error)
print(error)
check_error_point_etl(t_id)

In [ ]:
#df_vInfo.to_excel('vinfo.xlsx')

In [ ]:
df_vInfo.info()

# Transform VM Info

In [ ]:
print("Filter and Transform VM Info")

try:

    
 df_vInfo['month']=month_param
 df_vInfo['year']=year_param
 df_vInfo['import_date']=x_datenow


 df_vInfo['memory']=df_vInfo['memory'].apply(vm_util.set_size_gb)


 df_vInfo['capacity_gb']=df_vInfo['capacity_gb'].apply(vm_util.set_size_gb)

 df_vInfo['database_no_cal'] = df_vInfo['database_no_cal'].apply(pd.to_numeric, errors='coerce')     
 df_vInfo= df_vInfo.replace( {'database_no_cal':{np.nan:0}}) 

# add transaction ID
 df_vInfo['transaction_id']= t_id
    
 # replace nan with None then call replace_x_character  


 df_vInfo=df_vInfo.where(pd.notnull(df_vInfo), None)

 
    
 df_vInfo['system_name']=df_vInfo['system_name'].apply(vx.replace_x_character)
 df_vInfo['cost_center']=df_vInfo['cost_center'].apply(vx.replace_x_character)   
 df_vInfo['database']=df_vInfo['database'].apply(vx.replace_x_character) 
 

except Exception as err:
  list_error.append(True)
  error_message= str(err)
  print(error_message)   
  vm_util.add_error_to_database(14,error_message,t_id)  
  

    
check_error_point_etl(t_id)



vinfo_ok=True

In [ ]:
print("VInfo data completed")
print(df_vInfo.info())
print(df_vInfo[['vm','powerstate','created_date','terminated_date','import_date','month','year']])

check_error_point_etl(t_id)

# End up with  Completed ETL in only Checking Data mode

In [ ]:
if is_only_check_data==True:
    try: 
        for file in  [disk_filepath ,info_filepath]:
         delete_resultList= fd_mn.delele_file(file)   
        
        updated_rows=vm_util.created_transaction(t_id)
        print("completed ETL in only Checking Data mode")
        
        exit()
        
        
    except Exception as ex:
        list_error.append(True)
        print(ex)
        raise Exception("Program is teminated and check error from email and log_error.txt")
    


# Create additional dynamic field/column by json

In [ ]:
print("create active additiona dynamic field/column by json")
extraFieldList=df_extra_mappingFields['column_table_name'].tolist()
print(extraFieldList)

In [ ]:

def add_additional_field_json(row):
    dict_fileds={}
    
    for key in extraFieldList: 
        valx=row[key]
        nan_valx=pd.isnull(valx)
        if  (nan_valx==False):
            dict_fileds[key]=valx
            
#     if len(dict_fileds.keys())==0 :
#      #df.replace('N/A',np.NaN)
#      return None
#      # return {}
#     else   : 
    return json.dumps(dict_fileds)

   
df_vInfo['additional_info']=df_vInfo.apply(add_additional_field_json,axis=1)


In [ ]:
print(df_vInfo.head())
print(df_vInfo.info())
#df_vInfo.to_excel('vm.xlsx',index=False)

# Add data to table

In [ ]:
print("Add datafrome to database")
#get table name from key value
table_name,listCols_report_vInfo,is_error=vx.listCols_report_table(vm_tb_key,t_id)
list_error.append(is_error)

check_error_point_etl(t_id)
print(f'list all columns in {table_name} table: ',len(listCols_report_vInfo))
print(listCols_report_vInfo)

# test_remove={}
# mostwanted_col = [ele for ele in dfCols_report_vInfo[column_name]
#                   if ele not in test_remove] 
# print('mostwanted_col:',mostwanted_col)
# listCols_report_vInfo=mostwanted_col

print('+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
listCols_dfx=df_vInfo.columns.tolist()
print(f'list all columns in {table_name} dataframe: ',len(listCols_dfx))
print(listCols_dfx)


In [ ]:
check_columns_2=vm_util.verify_existing_columns(listCols_dfx,listCols_report_vInfo)

print(check_columns_2)
listCols_report_vInfo

In [ ]:
if check_columns_2  is None:
   df_vInfo=df_vInfo[listCols_report_vInfo]
   print(df_vInfo.info())
   #df_vInfo.to_excel('report_vInfo_before_DB.xlsx')
   print(df_vInfo.head())
   rslt=db_command.add_data_values(db_command.get_postgres_conn(),df_vInfo,table_name)
   #print(rslt)
    
else:
   list_error.append(True)
   error_message=check_columns_2
   vm_util.add_error_to_database(10,error_message,t_id)
   print(error_message)

In [ ]:
check_error_point_etl(t_id)

# Update new cost center

In [ ]:
print("Update new cost center")

try :
 cost_centerList = df_vInfo['cost_center'].unique()
 df_new_cc=vm_util.add_new_cost_center(cost_centerList,'VM-ETL',t_id)
 print(df_new_cc)   
except Exception as ex:
 list_error.append(True)   
 print(ex)   

check_error_point_etl(t_id)

# Upate new addtional cost(category)

In [ ]:
print("Upate new addtional cost(category)")
df_vInfo=vm_util.create_extra_fields_byJson(df_vInfo)
df_vInfo=df_vInfo.where(pd.notnull(df_vInfo), None)

print("List addtional cost type for update")
print(df_vInfo.info())

#df_vInfo.to_excel("report_vInfo_afterAddCate_DB.xlsx")


In [ ]:
new_cost=[]
def collect_new_addtional_to_sent_mail(df_new_cost,tt_id):
#'CostID', 'CostName','CateID','CateName'  
 try:
     link_update_price=vm_util.get_config_value(update_additional_cost_key, tt_id)
     link_all_price=vm_util.get_config_value(all_additional_cost_key, tt_id)   
     content_data_dict= {
                     "ContentTitle":"List New Addional Cost",
                     "New_Additional_Cost":df_new_cost,
                     "Len_Cols_New_Additional_Cost": len(df_new_cost.columns),
                     'LinkAll':link_all_price,
                     'LinkEach':link_update_price
                    } 
     ok= x_mail.send_email(email_type='new_additional_cost',transaction_id=tt_id,
                     attached_file_path=None,content_data_dict=content_data_dict)  
     return ok   
    
 except Exception as ex:
     print(str(ex))   
     list_error.append(True)
      
    

In [ ]:
def new_category_cost(cost_name,cate_column_id,cate_column_name):
 # sql = "select cost_name from master_additional_cost  where  id =1"
 
 cost_id =0;
 try  : 
        
    if (cost_name is not None) and (cost_name !=unknow_val) :
     cost_name=str(cost_name)   
     #print('xyz: ',cate_column_id,'-',cost_name)
     x=cost_name.replace(' ','').lower()
     sql="select cost_name from master_additional_cost  where cost_column_id = (%s) and lower(replace(cost_name, ' ', '')) = (%s)"
     params=(cate_column_id,x)
     cost_item=db_command.get_one_sql(db_command.get_postgres_conn(),sql,params)
     #print('type-x',type(cost_item))
     
     if(cost_item is None):
       #print('new-item : ',cost_item,' as ',type(cost_item))     
       params = (cate_column_id, cost_name,cost_name)
       sql_insert="""INSERT INTO master_additional_cost(cost_column_id,cost_name,description)
             VALUES(%s,%s,%s) RETURNING id;"""
       cost_id=db_command.add_one_data_sql(db_command.get_postgres_conn(),sql_insert,params)
       new_cost.append([cost_id,cost_name,cate_column_name,cost_name]) 
    
 except Exception as err:
    list_error.append(True)
    error_message= str(err)
    print(error_message)   
    vm_util.add_error_to_database(12,error_message,t_id)   
    

In [ ]:
try:
    sql="select * from datafield_mapping where id in( select distinct on (cost_column_id) cost_column_id from master_additional_cost UNION SELECT id from datafield_mapping where is_additional=True and is_active=True)"

    print("List category cost for update data")
    list_cate_cost=db_command.get_list_sql(db_command.get_postgres_conn(),sql,None)
    df_cateCost=None
    if list_cate_cost is  not None:
     df_cateCost=pd.DataFrame(list_cate_cost)
     print(df_cateCost)  

except Exception as ex:
    list_error.append(True)
    print(str(ex))   
    vm_util.add_error_to_database(12,str(ex),t_id)  


In [ ]:
#new_category_cost(cate_column_id,cost_name):

if df_cateCost is not None :
 for index,row  in  df_cateCost.iterrows():
      cate_column_id=row['id']
      costtype_name=row['column_table_name']
      if costtype_name in listCols_dfx: # update only additional type in df columns 
         print("=============================================")
         #print(cate_column_id,'-',costtype_name)
         #print(df_vInfo[costtype_name])
         df_vInfo[costtype_name].apply(new_category_cost,args=(cate_column_id,costtype_name))
            
  

# How to get new additonal cost (cost type and cost anme ) in order to notify admin for adding price 

In [ ]:
if len(new_cost)>0:
 df_new_additonal_cost = pd.DataFrame(new_cost, columns = ['CostID', 'CostName','CateName','description'])
 print(df_new_additonal_cost) 
 collect_new_addtional_to_sent_mail(df_new_additonal_cost,t_id)    
  

In [ ]:
 check_error_point_etl(t_id)

# Completed ETL

In [ ]:
# veeam_ok=True  veeam_filepath
# vdisk_ok=True  disk_filepath
# vinfo_ok=True  info_filepath
#happy case
try: 
    #etl_files=[[veeam_ok,veeam_filepath],[vdisk_ok,disk_filepath] ,[vinfo_ok,info_filepath]]
    
    # disable veeam
    etl_files=[[vdisk_ok,disk_filepath] ,[vinfo_ok,info_filepath]]
    
    if veeam_ok==True and vdisk_ok==True  and vinfo_ok==True:
     vm_util.finished_etl_folder(t_id,vm_col_key,True,etl_files)    
     updated_rows=vm_util.created_transaction(t_id)
     print("completed ETL")
    
except Exception as ex:
    list_error.append(True)
    print(ex)
    


In [ ]:
check_error_point_etl(t_id)